In [ ]:
"""A very simple MNIST classifier.
See extensive documentation at
https://www.tensorflow.org/get_started/mnist/beginners
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys

from tensorflow.examples.tutorials.mnist import input_data

import tensorflow as tf

FLAGS = None


我们在这里调用系统提供的Mnist数据函数为我们读入数据，如果没有下载的话则进行下载。

<font color=#ff0000>**这里将data_dir改为适合你的运行环境的目录**</font>

In [ ]:
# Import data
data_dir = '/tmp/tensorflow/mnist/input_data'
mnist = input_data.read_data_sets(data_dir, one_hot=True)

一个非常非常简陋的模型

In [ ]:
# Create the model
x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
y = tf.matmul(x, W) + b

定义我们的ground truth 占位符

In [ ]:
# Define loss and optimizer
y_ = tf.placeholder(tf.float32, [None, 10])

接下来我们计算交叉熵，注意这里不要使用注释中的手动计算方式，而是使用系统函数。
另一个注意点就是，softmax_cross_entropy_with_logits的logits参数是**未经激活的wx+b**

In [ ]:
# The raw formulation of cross-entropy,
#
#   tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(tf.nn.softmax(y)),
#                                 reduction_indices=[1]))
#
# can be numerically unstable.
#
# So here we use tf.nn.softmax_cross_entropy_with_logits on the raw
# outputs of 'y', and then average across the batch.
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

生成一个训练step

In [ ]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

sess = tf.Session()
init_op = tf.global_variables_initializer()
sess.run(init_op)

在这里我们仍然调用系统提供的读取数据，为我们取得一个batch。
然后我们运行3k个step(5 epochs)，对权重进行优化。

In [ ]:
# Train
for _ in range(3000):
  batch_xs, batch_ys = mnist.train.next_batch(100)
  sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

验证我们模型在测试数据上的准确率

In [ ]:
  # Test trained model
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: mnist.test.images,
                                      y_: mnist.test.labels}))

毫无疑问，这个模型是一个非常简陋，性能也不理想的模型。目前只能达到92%左右的准确率。
接下来，希望大家利用现有的知识，将这个模型优化至98%以上的准确率。
Hint：
- 卷积
- 池化
- 激活函数
- 正则化
- 初始化
- 摸索一下各个超参数
  - 卷积kernel size
  - 卷积kernel 数量
  - 学习率
  - 正则化惩罚因子
  - 最好每隔几个step就对loss、accuracy等等进行一次输出，这样才能有根据地进行调整